<a href="https://colab.research.google.com/github/YKulki/DE_HUL/blob/main/DE_ACGIH_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files  #to upload a local file into stream
uploaded = files.upload()

Saving DE Input File - Sheet1.csv to DE Input File - Sheet1.csv


In [ ]:
import io                       #to read uploaded input file according to specification
import pandas as pd             #pandas used to read write data into csv

df = pd.read_csv(io.BytesIO(uploaded['DE Input File - Sheet1.csv']))            #For Calculation Basis and code understanding, please refer to the report 
df['Tag No.'] = df['Tag No.'].astype('string')
df['Powder Type'] = df['Powder Type'].astype('string')
df['Belt Drop Heights Inlet(mm)'] = df['Belt Drop Heights Inlet(mm)'].astype('string')
df['Outlet Areas(m2)'] = df['Outlet Areas(m2)'].astype('string')
df['Opening Area(m2)'] = df['Opening Area(m2)'].astype('string')                        #conversions of values into strings to read easily

In [ ]:
def belt_conveyor(length,width,speed,inlets,outlets,drop_heights,areas,powder_flow_rate,dust_gen_factor):   

  inlet_q=[]            #three hood flow rate lists
  length_q=[]
  outlet_q=[]


  #emp_factor=float(input("Material dust correction factor"))

  width=0.00328084*width #in mm  
  speed=196.85*speed  #in m/s


  length=3.28084*length #in m
  heights=list(map(float, drop_heights.split(',')))


  for x in range(inlets):                   #ACGIH Method for Inlet
    drop_height=heights[x]
    if drop_height<0:
      inlet_q.append(0)
    else:
      if drop_height<0.9144:
        if speed <200:
          inlet_q.append(594.6536*width)
        else:
          inlet_q.append(849.5051*width)
      else:
        if speed<200:
          inlet_q.append(594.6536*width)
        else:
          inlet_q.append(849.5051*width)
        if width<3:
          inlet_q.append(1189.3071)
        else:
          inlet_q.append(1699.0102)


  n=int(length/30)                        #ACGIH for Length Hoods - read document sample calculations
  for x in range(n):
    length_q.append(594.6536*width)

  outlet_areas=list(map(float, areas.split(',')))

  for x in range(outlets):                #ACGIH for Outlets
    outlet_area=10.7639*outlet_areas[x]   #in m2
    outlet_q.append(339.802*outlet_area)

  
  q_powder_flow=dust_gen_factor*powder_flow_rate    #Method 3 sense check

  if sum(inlet_q)+sum(length_q)+sum(outlet_q)<q_powder_flow:
    print("Check with emperical formula")
  else:  
    Inlet_Q = ",".join(map(str, inlet_q))
    Outlet_Q =",".join(map(str, outlet_q))
    Length_Q =",".join(map(str, length_q))



    return [Inlet_Q,Outlet_Q,Length_Q]      #Returning values

In [ ]:
def vessel(openings,volume,hopper_type,cycles,empty_time,opening_area,powder_flow_rate,dust_gen_factor):

  q_calc=[]

  opening_area=list(map(float, opening_area.split(',')))

  if volume>0:                        #Method 4 for batch and discharge hoppers
    if hopper_type=='B':
      q1=dust_gen_factor*volume*cycles #cycles per hr,empty time in s
      if empty_time>0:
        q2= dust_gen_factor*volume*3600*10/empty_time
      else:
        q2=0
      q_airchange=max(q1,q2)
    elif hopper_type=='D':
      #8-15 air changes recommended
      q1=dust_gen_factor*volume*cycles #cycles per hr,empty time in s
      q_airchange=q1
    else:
      print("Incorrect input")
  else: 
    q_airchange=0

  for x in range(openings):               #Method 2 - face velocity =1m/s
    if opening_area[x]>0:
      area=dust_gen_factor*10.7639*opening_area[x] #in m^2
      q_calc.append(339.8*area)
    else:
      q_calc.append(0)

  q_powder_flow=dust_gen_factor*float(15*powder_flow_rate)    #Method 3

  return max(sum(q_calc),q_airchange,q_powder_flow)

In [ ]:
 def Sieve(sieve_area):

  sieve_area=10.7639*sieve_area

  q=1.699*50*sieve_area             #ACGIH Method

  q_powder_flow=float(20*dust_gen_factor*powder_flow_rate)    #Method 3

  return max(q,q_powder_flow)

In [ ]:
def LIW(width,areas,powder_flow_rate,dust_gen_factor,volume,empty_time):
  
  if areas>0:
    outlet_area=10.7639*areas   #in m2
    outlet_q=(339.802*outlet_area)          #For conveyor attached to LIWF - ACGIH based
  else:
    outlet_q=0

  q_oneairchange=15*volume/empty_time*dust_gen_factor*3600    #Method 4 for vessel of LIWF
  q_powder_flow=float(dust_gen_factor*powder_flow_rate*15)    #Method 3 Sense check

  q_calc=max(q_oneairchange,q_powder_flow)


  return [outlet_q,q_calc]

In [ ]:
def Boon(powder_flow_rate,dust_gen_factor):
  q=powder_flow_rate*dust_gen_factor*20
  if q>2720:
    Boon_q=[q,q]
  else:
    Boon_q='1359.2081,1359.2081'            #ACGIH Method
  return Boon_q

In [ ]:
q=[]

for ind in df.index:                            #Execution Cell for the Code - Reads and Writes into File after calling each function
  if not pd.isnull(df.loc[ind, 'Tag No.']): 
    if df['Tag No.'][ind][:2]== "BC" :
      length=df['Length(m)'][ind]
      width=df['Belt Width(mm)'][ind]
      speed=float(df['Belt Speed (m/s)'][ind])
      inlets=int(df['Inlets'][ind])
      outlets=int(df['Outlets'][ind])
      drop_heights=df['Belt Drop Heights Inlet(mm)'][ind]
      areas=df['Outlet Areas(m2)'][ind]
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      q=(belt_conveyor(length,width,speed,inlets,outlets,drop_heights,areas,powder_flow_rate,dust_gen_factor)) #Function call for Belt conveyor
    elif df['Tag No.'][ind][:2]== "WH"or df['Tag No.'][ind][:2]== "DH" or df['Tag No.'][ind][:2]== "VS" :
      openings=int(df['Openings'][ind])
      opening_area=df['Opening Area(m2)'][ind] 
      volume=df['Volume(m3)'][ind]
      hopper_type=df['Hopper Type'][ind]
      cycles=df['Cycles (per hour)'][ind]
      empty_time=df['Empty Time(s)'][ind]
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      q=vessel(openings,volume,hopper_type,cycles,empty_time,opening_area,powder_flow_rate,dust_gen_factor)   #Function call for Vessels
    elif df['Tag No.'][ind][:1]=="S":
      sieve_area=float(df['Sieve_Area(m2)'][ind])
      q=Sieve(sieve_area)       #Function call for Sieves
    elif df['Tag No.'][ind][:1]=="X":
      width=df['Belt Width(mm)'][ind]
      areas=float(df['Outlet Areas(m2)'][ind])
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      volume=df['Volume(m3)'][ind]
      empty_time=df['Empty Time(s)'][ind]
      q=LIW(width,areas,powder_flow_rate,dust_gen_factor,volume,empty_time)     #Function call for LIWF
    elif df['Tag No.'][ind][:2]=="MX":
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      q=Boon(powder_flow_rate,dust_gen_factor)        #Function call for Mixers
    else:
      print("Soz")
  if df['Tag No.'][ind][:2]== "BC":         #Writing into CSV
    df['Inlet_Q'][ind]=q[0]
    df['Outlet_Q'][ind]=q[1]
    df['Length_Q'][ind]=q[2]
  elif df['Tag No.'][ind][:2]== "WH"or df['Tag No.'][ind][:2]== "DH" or df['Tag No.'][ind][:2]== "VS":
    df['Vessel_Q'][ind]=q
  elif df['Tag No.'][ind][:1]=="S":
    df['Sieve_Q'][ind]=q
  elif df['Tag No.'][ind][:1]=="X":
    df['Outlet_Q'][ind]=q[0]
    df['Vessel_Q'][ind]=q[1]
  elif df['Tag No.'][ind][:2]=="MX":
    df['Mixer_Q'][ind]=q
  else:
    print("Component Not Present, Please Check!")  #For components not present
      




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

In [ ]:
df.to_csv('Output File.csv')            #To donwload Result File
files.download('Output File.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>